# Système Multi-Agents avec Q-Learning pour l'Ordonnancement des Patients

Ce notebook démontre l'implémentation d'un système multi-agents pour l'optimisation de l'ordonnancement des patients dans un environnement de soins.

## Caractéristiques principales:
- **Métaheuristiques hybrides**: Algorithme Génétique, Recherche Tabou, Recuit Simulé
- **Modes de collaboration**: Amis (partage complet) et Ennemis (compétition)
- **Auto-adaptation**: Q-Learning pour la sélection des voisinages
- **Diversité**: Espace Mémoire Partagé (EMP) avec contrôle de distance
- **5 fonctions de voisinage**: A, B, C, D, E (C et E actifs pour les contraintes strictes)

Basé sur le diaporama: *"Optimisation collaborative : Agents auto-adaptatifs, Apprentissage par renforcement"*

In [ ]:
# Configuration et imports
import sys
import os
import random
import numpy as np
import matplotlib.pyplot as plt

# Ajout du chemin courant
sys.path.insert(0, os.path.abspath(os.getcwd()))

# Seed pour reproductibilité
random.seed(42)
np.random.seed(42)

# Imports du projet
from core.environment import create_default_environment, Task
from core.neighborhoods import NeighborhoodManager
from core.qlearning import QLearningAgent
from core.shared_memory import SharedMemoryPool, Solution, ElitePool
from core.agents import (
    GeneticAgent, TabuAgent, SimulatedAnnealingAgent,
    MultiAgentSystem, CollaborationMode
)
from visualization import plot_gantt

print("✓ Tous les modules importés avec succès!")

## 1. Environnement d'Ordonnancement

L'environnement modélise l'ordonnancement des patients avec:
- 10 patients
- 5 opérations maximum par patient
- 6 compétences/ressources médicales

In [ ]:
# Créer l'environnement
env = create_default_environment()

print(f"Configuration de l'environnement:")
print(f"  - Nombre de patients: {env.num_patients}")
print(f"  - Compétences: {env.skills}")
print(f"  - Opérations max par patient: {env.max_ops}")
print(f"  - Nombre total de tâches: {len(env.all_tasks)}")

# Solution initiale
initial_solution = env.build_initial_solution(random_order=False)
initial_makespan, initial_times, _ = env.evaluate(initial_solution, return_schedule=True)
print(f"\nMakespan initial (ordre naïf): {initial_makespan}")

In [ ]:
# Visualiser le planning initial
if initial_times:
    plot_gantt(
        initial_times, env.skills, env.num_patients,
        title=f"Planning Initial (Cmax = {initial_makespan})"
    )

## 2. Fonctions de Voisinage

Les voisinages actifs pour ce problème strict sont principalement :
- **C**: Insertion dans le même planning (MIS)
- **E**: Échange dans le même planning (SSMS)

In [ ]:
# Démonstration des voisinages
nm = NeighborhoodManager()

print("Fonctions de voisinage disponibles:")
for name, func in nm.neighborhoods.items():
    print(f"  {name}: {func.name}")

# Générer des voisins pour chaque fonction valide
test_solution = env.build_initial_solution(random_order=True)
test_makespan, _, _ = env.evaluate(test_solution)
print(f"\nSolution de test: Makespan = {test_makespan}")

print("\nGénération de voisins (sur voisinages actifs C & E):")
for name in ['C', 'E']:
    neighbor = nm.generate_neighbor(test_solution, name, env.skills, env.max_ops)
    if neighbor:
        neighbor_makespan, _, _ = env.evaluate(neighbor)
        diff = neighbor_makespan - test_makespan
        print(f"  Voisinage {name}: Makespan = {neighbor_makespan} (Δ = {diff:+d})")

## 3. Q-Learning pour l'Auto-Adaptation

Le Q-Learning permet de sélectionner automatiquement la meilleure fonction de voisinage.

In [ ]:
# Créer un agent Q-Learning
q_agent = QLearningAgent(
    states=['C', 'E'],
    alpha=0.15,    # Taux d'apprentissage
    gamma=0.9,     # Facteur d'actualisation
    epsilon=0.5,   # Exploration initiale
    epsilon_decay=0.99
)

print("Paramètres Q-Learning:")
print(f"  α (learning rate): {q_agent.alpha}")
print(f"  γ (discount factor): {q_agent.gamma}")
print(f"  ε (exploration): {q_agent.epsilon}")

In [ ]:
# Simulation d'apprentissage simple
reward_means = {'C': 1.0, 'E': 0.5}
epsilon_history = []

for episode in range(100):
    action = q_agent.select_action()
    reward = random.gauss(reward_means[action], 0.2)
    q_agent.update(action, reward)
    q_agent.decay_epsilon()
    epsilon_history.append(q_agent.epsilon)

# Visualiser la table Q (Affichage textuel)
q_table = q_agent.get_q_table_formatted()
print("\nTable Q après apprentissage (Scores par transition):")
for s_from, actions in q_table.items():
    print(f"État {s_from} -> {actions}")

## 4. Espace Mémoire Partagé (EMP) avec Diversité

L'EMP stocke les bonnes solutions trouvées tout en maintenant leur diversité.

In [ ]:
# Créer l'EMP
emp = SharedMemoryPool(
    max_size=15,
    min_distance=2,      # R: distance minimale
    diversity_threshold=0.4  # DT: seuil de diversité
)

print("Paramètres EMP:")
print(f"  Taille max: {emp.max_size}")
print(f"  Distance minimale (R): {emp.min_distance}")

# Remplir l'EMP avec quelques solutions aléatoires
for i in range(20):
    sol = env.build_initial_solution(random_order=True)
    fitness, _, _ = env.evaluate(sol)
    solution = Solution(sequences=sol, fitness=fitness, agent_id=f"test_{i}")
    emp.insert(solution, iteration=i)

stats = emp.get_statistics()
print(f"\nStatistiques EMP:")
print(f"  Solutions stockées: {stats['size']}")
print(f"  Meilleure fitness: {stats['global_best_fitness']}")
print(f"  Insertions: {stats['insertions']}")
print(f"  Rejets (doublons): {stats['rejections_duplicate']}")
print(f"  Rejets (diversité): {stats['rejections_diversity']}")

## 5. Système Multi-Agents - Mode AMIS

En mode Amis, les agents partagent leurs solutions complètes via l'EMP.

In [ ]:
# Créer le système multi-agents en mode Amis
random.seed(42)
np.random.seed(42)

mas_friends = MultiAgentSystem(env, mode=CollaborationMode.FRIENDS, use_qlearning=True)

# Ajouter des agents de différents types
mas_friends.add_agent('genetic', 'AG_1', population_size=15)
mas_friends.add_agent('tabu', 'Tabu_1', tabu_tenure=10)
mas_friends.add_agent('sa', 'RS_1', initial_temp=100)

print(f"Mode: {mas_friends.mode}")
print(f"Agents: {list(mas_friends.agents.keys())}")

In [ ]:
# Exécuter l'optimisation
print("Optimisation collaborative en cours...")
best_solution_friends = mas_friends.run(n_iterations=50, verbose=True)

stats_friends = mas_friends.get_statistics()
print(f"\nMeilleur Makespan trouvé: {stats_friends['global_best_fitness']}")

In [ ]:
# Visualiser la convergence (Extraction manuelle de l'historique d'un agent pour l'exemple)
history_friends = list(mas_friends.agents.values())[0].fitness_history

plt.figure(figsize=(10, 5))
plt.plot(history_friends, label="AG_1 History")
plt.title("Convergence (Exemple Agent AG_1 - Mode Amis)")
plt.xlabel("Itérations")
plt.ylabel("Makespan")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Visualiser le planning optimisé
if best_solution_friends:
    final_makespan, final_times, _ = env.evaluate(
        best_solution_friends.sequences, return_schedule=True
    )
    
    plot_gantt(
        final_times, env.skills, env.num_patients,
        title=f"Résultat Optimisé (Mode Amis, Cmax={final_makespan})"
    )

## 6. Système Multi-Agents - Mode ENNEMIS

En mode Ennemis, les agents ne partagent que leurs valeurs de fitness.

In [ ]:
# Créer le système en mode Ennemis
random.seed(42)
np.random.seed(42)

mas_enemies = MultiAgentSystem(env, mode=CollaborationMode.ENEMIES, use_qlearning=True)

mas_enemies.add_agent('genetic', 'AG_Enemy')
mas_enemies.add_agent('tabu', 'Tabu_Enemy')
mas_enemies.add_agent('sa', 'RS_Enemy')

print(f"Mode: {mas_enemies.mode}")
print("\nEn mode Ennemis, les agents sont en compétition.")

In [ ]:
# Exécuter l'optimisation
print("\nOptimisation compétitive en cours...")
best_solution_enemies = mas_enemies.run(n_iterations=50, verbose=True)

stats_enemies = mas_enemies.get_statistics()
print(f"\nMeilleur Makespan trouvé: {stats_enemies['global_best_fitness']}")

## 7. Comparaison Finale

In [ ]:
# Tableau de comparaison
print("="*60)
print("        COMPARAISON DES MODES DE COLLABORATION")
print("="*60)
print(f"{'Métrique':<35} {'AMIS':>12} {'ENNEMIS':>12}")
print("-"*60)
print(f"{'Meilleur Makespan':<35} {stats_friends['global_best_fitness']:>12} {stats_enemies['global_best_fitness']:>12}")
print(f"{'Taille finale EMP':<35} {stats_friends['emp_stats']['size']:>12} {stats_enemies['emp_stats']['size']:>12}")
print(f"{'Insertions EMP':<35} {stats_friends['emp_stats']['insertions']:>12} {stats_enemies['emp_stats']['insertions']:>12}")

improvement_friends = (initial_makespan - stats_friends['global_best_fitness']) / initial_makespan * 100
improvement_enemies = (initial_makespan - stats_enemies['global_best_fitness']) / initial_makespan * 100
print(f"{'Amélioration vs initial (%)':<35} {improvement_friends:>11.1f}% {improvement_enemies:>11.1f}%")
print("="*60)

In [ ]:
# Planning final optimisé
if stats_friends['global_best_fitness'] <= stats_enemies['global_best_fitness']:
    best_overall = best_solution_friends
    best_mode = "AMIS"
else:
    best_overall = best_solution_enemies
    best_mode = "ENNEMIS"

print(f"\nMeilleure solution globale trouvée par le mode {best_mode}")

final_makespan, final_times, _ = env.evaluate(best_overall.sequences, return_schedule=True)

plot_gantt(
    final_times, env.skills, env.num_patients,
    title=f"Planning Optimisé Final (Mode {best_mode}, Cmax = {final_makespan})"
)

## Conclusion

Ce système multi-agents démontre comment combiner:

1. **Plusieurs métaheuristiques** (AG, Tabou, Recuit) pour diversifier la recherche
2. **Deux modes de collaboration** (Amis/Ennemis) pour différentes stratégies
3. **L'auto-adaptation via Q-Learning** pour choisir intelligemment les voisinages
4. **Le contrôle de diversité** dans l'EMP pour éviter la convergence prématurée